<a href="https://colab.research.google.com/github/nachospimp/pf/blob/main/TA1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sat Aug 10 13:26:53 2024

@author: giovannamontalvo
"""

import pandas as pd
import numpy as np
import yfinance as yf
import pandas_ta as ta
import plotly.io as pio
pio.renderers.default = 'browser'
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Descargar datos de Yahoo Finance
ticker = yf.Ticker('AAPL')
data = ticker.history(interval="1d", period="5y")

# Universo IPC
# # BIMBOA.MX CEMEXCPO.MX, ALSEA.MX, GMEXICOB.MX FEMSAUBD.MX, FIBRAPL14.MX, GFNORTEO.MX, AMXB.MX

# Leer datos desde un archivo CSV
# Asegúrate de que el archivo CSV tenga las columnas: Date, Open, High, Low, Close, Volume
#csv_file = 'CEMEXCPO_1y.csv'
#data = pd.read_csv(csv_file)

# Change 'Date' below to the actual name of the date column if needed
#data['Date'] = pd.to_datetime(data['Date']) #Datetime
#data.set_index('Date', inplace=True)

# Calcular indicadores: VWAP, RSI, Bandas de Bollinger
data["VWAP"] = ta.vwap(data.High, data.Low, data.Close, data.Volume)
data["RSI"] = ta.rsi(data.Close, length=16)
my_bbands = ta.bbands(data.Close, length=14, std=2.0)
data = data.join(my_bbands)

# Add suffixes to differentiate the overlapping columns
#data = data.join(my_bbands, lsuffix='_left', rsuffix='_right')

# Definir señales VWAPSignal y TotalSignal
VWAPsignal = [0] * len(data)
backcandles = 15

data['Max_Open_Close'] = np.maximum(data['Open'], data['Close'])
data['Min_Open_Close'] = np.minimum(data['Open'], data['Close'])
data['upt'] = 1
data['dnt'] = 1
data.loc[(data['Max_Open_Close'] >= data['VWAP']), 'dnt'] = 0
data.loc[(data['Min_Open_Close'] <= data['VWAP']), 'upt'] = 0
data['sig_dnt'] = data['dnt'].rolling(backcandles + 1, min_periods=1).min()
data['sig_upt'] = data['upt'].rolling(backcandles + 1, min_periods=1).min()
data['VWAPSignal2'] = 0
data.loc[(data['sig_upt'] == 1) & (data['sig_dnt'] == 1), 'VWAPSignal2'] = 3
data.loc[(data['sig_upt'] == 1) & (data['sig_dnt'] == 0), 'VWAPSignal2'] = 2
data.loc[(data['sig_upt'] == 0) & (data['sig_dnt'] == 1), 'VWAPSignal2'] = 1

data['VWAPSignal'] = VWAPsignal

# Definir TotalSignal
def TotalSignal(row):
    if (data.VWAPSignal[row] == 2
        and data.Close[row] <= data['BBL_14_2.0'][row]
        and data.RSI[row] < 45):
        return 2

    if (data.VWAPSignal[row] == 1
        and data.Close[row] >= data['BBU_14_2.0'][row]
        and data.RSI[row] > 55):
        return 1

    return 0

TotSignal = [0] * len(data)
for row in range(backcandles, len(data)):
    TotSignal[row] = TotalSignal(row)

data['TotalSignal'] = TotSignal


# Calcular medias móviles simples (SMA)
data.ta.sma(length=20, append=True) # Medias en minutos. Ej. 20 min avg.
data.ta.sma(length=30, append=True) # Medias en minutos. Ej. 20 min avg.
data.ta.sma(length=50, append=True)
data.ta.sma(length=200, append=True)

# Calcular medias móviles simples (SMA) para días
#minutes_per_day = 1440
#sma_periods = {
 #   'SMA_20': 20 * minutes_per_day,
  #  'SMA_50': 50 * minutes_per_day,
   # 'SMA_200': 200 * minutes_per_day
#}

#for label, period in sma_periods.items():
 #   data[label] = data['Close'].rolling(window=period, min_periods=1).mean()

# Crear una columna para los puntos de señal de compra basada en RSI < 30
data['LowSignal_RSI'] = np.where(data['RSI'] < 20, data['Low'], np.nan)

# Crear una columna para los puntos de señal de compra basada en RSI > 70
data['HighSignal_RSI'] = np.where(data['RSI'] > 80, data['High'], np.nan)

# Crear una columna para los puntos de señal
data['pointposbreak'] = np.where(data['TotalSignal'] != 0, data['High'] + 1e-4, np.nan)

# Crear una columna para los punto de psible tendencia alcista o bajista tras los cruces de las SMA 20 y 50
data['SMA_20_50_Bullish'] = np.where((data['SMA_20'] > data['SMA_50']) & (data['SMA_20'].shift(1) <= data['SMA_50'].shift(1)), 1, 0)
data['SMA_20_50_Bearish'] = np.where((data['SMA_20'] < data['SMA_50']) & (data['SMA_20'].shift(1) >= data['SMA_50'].shift(1)), 1, 0)

# Agregar señales de entrada y salida
data['Entry'] = np.nan
data['Exit'] = np.nan

# Señales de entrada y salida basadas en la estrategia
for i in range(1, len(data)):
    if (data['Close'].iloc[i] <= data['BBL_14_2.0'].iloc[i]) and (data['RSI'].iloc[i] <= 30):
        data['Entry'].iloc[i] = data['Low'].iloc[i]
    elif (data['Close'].iloc[i] >= data['BBU_14_2.0'].iloc[i]) and (data['RSI'].iloc[i] >= 70):
        data['Exit'].iloc[i] = data['High'].iloc[i]


# Definir funciones para identificar soporte y resistencia
def is_support_level(data, i):
    support = data['Low'][i] < data['Low'][i - 1] and \
              data['Low'][i] < data['Low'][i + 1] and \
              data['Low'][i + 1] < data['Low'][i + 2] and \
              data['Low'][i - 1] < data['Low'][i - 2]
    return support

def is_resistance_level(data, i):
    resistance = data['High'][i] > data['High'][i - 1] and \
                 data['High'][i] > data['High'][i + 1] and \
                 data['High'][i + 1] > data['High'][i + 2] and \
                 data['High'][i - 1] > data['High'][i - 2]
    return resistance

# Limpiar niveles de soporte y resistencia
mean = np.mean(data['High'] - data['Low'])

def distance_from_level(x):
    return np.sum([abs(x - y[1]) < mean for y in levels]) == 0

levels = []
for i in range(2, data.shape[0] - 2):
    if is_support_level(data, i):
        x = data['Low'][i]
        if distance_from_level(x):
            levels.append((data.index[i], x))
    elif is_resistance_level(data, i):
        x = data['High'][i]
        if distance_from_level(x):
            levels.append((data.index[i], x))

# Seleccionar datos para graficar
st = 0
# Ensure st is large enough to include calculated Bollinger Bands
st = max(st, 14) # 14 is the length used for BBands calculation
#dfp1 = data.iloc[st:st+350]
dfp1 = data.copy()  # Esto incluirá todos los datos



# Identificar niveles de soporte y resistencia
levels = []
for i in range(2, dfp1.shape[0] - 2):
    if is_support_level(dfp1, i):
        levels.append((dfp1.index[i], dfp1['Low'][i]))
    elif is_resistance_level(dfp1, i):
        levels.append((dfp1.index[i], dfp1['High'][i]))

# Filtrar los datos donde el volumen es mayor que cero
dfp1 = dfp1[dfp1['Volume'] > 0]

# Asegurar que el índice sea de tipo datetime y ordenar por fecha
dfp1.index = pd.to_datetime(dfp1.index)
dfp1.sort_index(inplace=True)

# Imprimir los valores del RSI para los primeros 10 registros
print("Valores del RSI:")
print(dfp1['RSI'].tail(10))  # Muestra los primeros 10 valores del RSI

# Crear una columna para determinar el color del volumen
dfp1['VolumeColor'] = np.where(dfp1['Close'] > dfp1['Open'], 'green', 'red')

# Crear la figura en Plotly con subplots
fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    row_heights=[0.8, 0.25], vertical_spacing=0.025)

# Añadir el gráfico de velas
fig.add_trace(go.Candlestick(x=dfp1.index,
                             open=dfp1['Open'],
                             high=dfp1['High'],
                             low=dfp1['Low'],
                             close=dfp1['Close'],
                             increasing_fillcolor='black',
                             increasing_line_color='green',
                             decreasing_fillcolor='darkred',
                             decreasing_line_color='red',
                             name="Candlestick"),
              row=1, col=1)



# Añadir el gráfico de volumen con colores según el precio
fig.add_trace(go.Bar(x=dfp1.index, y=dfp1['Volume'],
                     marker=dict(color=dfp1['VolumeColor']),
                     showlegend=False, name='Volume'),
              row=2, col=1)

# Añadir indicadores al gráfico de velas
fig.add_trace(go.Scatter(x=dfp1.index, y=dfp1['VWAP'],
                         mode='lines', line=dict(color='blue', width=1),
                         name="VWAP"),
              row=1, col=1)

fig.add_trace(go.Scatter(x=dfp1.index, y=dfp1['BBL_14_2.0'],
                         mode='lines', line=dict(color='pink', width=1),
                         name="BBL"),
              row=1, col=1)

fig.add_trace(go.Scatter(x=dfp1.index, y=dfp1['BBM_14_2.0'],
                         mode='lines', line=dict(color='lightblue', width=1),
                         name="BBM"),
             row=1, col=1)

fig.add_trace(go.Scatter(x=dfp1.index, y=dfp1['BBU_14_2.0'],
                         mode='lines', line=dict(color='orange', width=1),
                         name="BBU"),
              row=1, col=1)

fig.add_trace(go.Scatter(x=dfp1.index, y=dfp1['SMA_20'],
                         mode='lines', line=dict(color='cyan', width=1),
                         name="SMA 20"),
             row=1, col=1)

fig.add_trace(go.Scatter(x=dfp1.index, y=dfp1['SMA_30'],
                         mode='lines', line=dict(color='darkorange', width=1),
                         name="SMA 30"),
             row=1, col=1)

fig.add_trace(go.Scatter(x=dfp1.index, y=dfp1['SMA_50'],
                        mode='lines', line=dict(color='purple', width=1),
                         name="SMA 50"),
             row=1, col=1)

#fig.add_trace(go.Scatter(x=dfp1.index, y=dfp1['SMA_200'],
 #                      mode='lines', line=dict(color='white', width=1),
  #                       name="SMA 200"),
   #           row=1, col=1)

# Añadir señales de entrada y salida
fig.add_trace(go.Scatter(x=dfp1.index, y=dfp1['Entry'],
                         mode='markers', marker=dict(size=10, color='blue', symbol='triangle-up'),
                         name='Entry Signal'),
              row=1, col=1)

fig.add_trace(go.Scatter(x=dfp1.index, y=dfp1['Exit'],
                         mode='markers', marker=dict(size=10, color='pink', symbol='triangle-down'),
                         name='Exit Signal'),
              row=1, col=1)

# Añadir señales de punto de ruptura y RSI
fig.add_trace(go.Scatter(x=dfp1.index, y=dfp1['pointposbreak'],
                         mode="markers", marker=dict(size=5, color="MediumPurple"),
                         name="Signal"),
              row=1, col=1)

fig.add_trace(go.Scatter(x=dfp1.index, y=dfp1['LowSignal_RSI'],
                         mode="markers", marker=dict(size=10, color="red", symbol="triangle-up"),
                         name="Signal (RSI < 20)"),
              row=1, col=1)

fig.add_trace(go.Scatter(x=dfp1.index, y=dfp1['HighSignal_RSI'],
                         mode="markers", marker=dict(size=10, color="green", symbol="triangle-up"),
                         name="Signal (RSI > 80)"),
              row=1, col=1)

# Anadir señales de tendencia alcista o bajista al gráfico
fig.add_trace(go.Scatter(x=data.index, y=np.where(data['SMA_20_50_Bullish'], data['High'] + 0.01, np.nan),
                         mode="markers", marker=dict(size=10, color="lime", symbol="triangle-up"), name="Bullish Crossover"),
              row=1, col=1)

fig.add_trace(go.Scatter(x=data.index, y=np.where(data['SMA_20_50_Bearish'], data['Low'] - 0.01, np.nan),
                         mode="markers", marker=dict(size=10, color="orange", symbol="triangle-down"), name="Bearish Crossover"),
              row=1, col=1)

# Agregar líneas de soporte y resistencia limpiadas al gráfico
for level in levels:
    fig.add_trace(go.Scatter(x=[level[0], dfp1.index[-1]],
                             y=[level[1], level[1]],
                             mode="lines",
                             line=dict(color="gray", width=1, dash="dash"),
                             showlegend=False,
                             visible=False,  # Asegura que las líneas sean visibles inicialmente
                             name=f"Nivel {i+1}"  # Etiqueta para cada línea de soporte/resistencia
                            ),
                  row=1, col=1) # Add the trace to the first row, first column

# Crear botones para mostrar/ocultar líneas de soporte y resistencia
updatemenus = [
    {
        'buttons': [
            {
                'label': 'Mostrar Soporte/Resistencia',
                'method': 'update',
                'args': [{'visible': [True] * len(fig.data)}, {'title': 'Gráfico con Soporte/Resistencia'}]
            },
            {
                'label': 'Ocultar Soporte/Resistencia',
                'method': 'update',
                'args': [{'visible': [True] * len(fig.data)}, {'title': 'Gráfico sin Soporte/Resistencia'}]
            }
        ],
        'direction': 'down',
        'showactive': False,
        'x': 0.1,  # Posición horizontal del menú
        'xanchor': 'left',  # Ancla del menú a la izquierda
        'y': 1.05,  # Posición vertical del menú
        'yanchor': 'top',  # Ancla del menú en la parte superior
        'pad': {'r': 10, 't': 10},  # Espaciado derecho y superior
        'bgcolor': 'rgba(255, 255, 255, 0.6)',  # Color de fondo con transparencia
        'bordercolor': 'rgba(0, 0, 0, 0.6)',  # Color del borde con transparencia
        'borderwidth': 2,  # Ancho del borde
    }
]

# Configurar diseño y mostrar la figura
fig.update_layout(
    #title='Gráfico de Velas con Indicadores y Señales',
    xaxis_title='Fecha',
    yaxis_title='Precio',
    height=1025,
    width=1700,
    template='plotly_dark', # Fondo oscuro del gráfico
    updatemenus=updatemenus
)


# Mostrar la figura
fig.show()